### Packages

In [36]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

import pandas as pd
import numpy as np

### Files to process

In [2]:
file_list = ["faces/celebrities/female/attractive/image_5.jpg",
            "faces/celebrities/female/attractive/image_16.jpg"]

### Function - enumerated landmarks

In [28]:
def total_function(picture_address):
    
    image = cv2.imread(picture_address)
    x = image.shape[0]
    y = image.shape[1]
    
    # part 1
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:
    
        image = cv2.imread(picture_address)
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # part 2
    string = str(results.multi_face_landmarks[0])
    string = string.replace('\nlandmark ', '').replace('\n  ', '').replace('\n  ', '').replace('x: ', '').replace('y: ', ', ')
    string = string.replace('z: ', ', ').replace('\n', '').replace('landmark', '').replace(' ', '')
    string = string.split('{')[1:]
    string = [item.replace('}', '') for item in string]
    landmark_list = []
    for item in string:
        sub_list = item.split(',')
        sub_list = [float(x) for x in sub_list]
        landmark_list.append(sub_list)
        
    # part 3 - it works, but for consistency and clarity, fix the 'x' and 'y' confusion
    df = pd.DataFrame(np.array(landmark_list))
    df = df.rename(columns = {0:'x', 1:'y', 2:'z'})
    
    df['x'] = df['x']*y
    df['y'] = df['y']*x
    
    df['x'] = df['x'].astype(int)
    df['y'] = df['y'].astype(int)
    
    
    # part 4
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (236, 135)
    fontScale = 0.2
    color = (255, 255, 255)
    thickness = 1
    
    index = 1
    for x_axis, y_axis in zip(df['x'],df['y']):
        cv2.circle(image,(x_axis, y_axis), 1,(255, 255, 255), thickness=1, lineType=cv2.LINE_8 )
        cv2.putText(image, str(index), (x_axis, y_axis), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.waitKey(0)
        index += 1
    
    cv2.imwrite('C:\\Users\\lewsz\\OneDrive\\Desktop\\face_landmarks\\TEST_5' + '.png', image)
    
    return df

### Function - regular landmarks

In [41]:
def regular_landmarks(file_address):
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    with mp_face_mesh.FaceMesh(static_image_mode=True,max_num_faces=1,min_detection_confidence=0.5) as face_mesh:
        image = cv2.imread(file_address)
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      mp_drawing.draw_landmarks(image=annotated_image, landmark_list=face_landmarks,
          connections=mp_face_mesh.FACE_CONNECTIONS,
          landmark_drawing_spec=drawing_spec,
          connection_drawing_spec=drawing_spec)
    cv2.imwrite('C:\\Users\\lewsz\\OneDrive\\Desktop\\face_landmarks\\landmark_image_1' + ".png", annotated_image)

### testing

In [29]:
total_function("faces/celebrities/female/attractive/image_16.jpg")

,x,y,z
0,252,426,-0.070509
1,252,378,-0.159374
2,252,391,-0.074463
3,240,319,-0.130050
4,252,362,-0.173274
...,...,...,...
463,297,268,-0.004492
464,288,273,-0.021317
465,282,276,-0.040682
466,376,252,0.011350


In [64]:
regular_landmarks("faces/celebrities/female/attractive/image_17.jpg")